In [2]:
pip install langchain-core langchain-google-vertexai pydantic elasticsearch --quiet


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from google.cloud import aiplatform

aiplatform.init(project="vertexllama", location="us-central1")

In [4]:
! gcloud auth login
! gcloud config set project vertexllama

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=fqshEL3ocX0rY8UfP9sSaO6R3XCN4M&access_type=offline&code_challenge=jWKRADIA2o3DeknygMtKWEVM_E1hXmNy-92GY0mbGJQ&code_challenge_method=S256


You are now logged in as [info@dataopslabs.com].
Your current project is [vertexllama].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID

To update your Application Default Credentials quota project, use the `gcloud auth application-default set-quota-project` command.
U

In [5]:
from langchain_google_vertexai import VertexAI

# To use model
model = VertexAI(model_name="gemini-pro")

In [6]:
# To specify a particular model version
model = VertexAI(model_name="gemini-1.0-pro-002")

In [7]:
message = "What are some of the pros and cons of Python as a programming language?"
model.invoke(message)

/opt/homebrew/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


'## Pros of Python:\n\n**1. Easy to learn and read:** Python has a simple and intuitive syntax, making it easier for beginners to learn and understand compared to other languages. Its code is often described as "readable like English," which makes it easier for developers to collaborate and maintain codebases.\n**2. Versatile and multi-purpose:** Python can be used for a wide range of tasks, including web development, data science, machine learning, scripting, automation, and more. This versatility makes it a valuable tool for developers with diverse interests and project needs.\n**3. Large and supportive community:** Python has a vast and active community of developers worldwide. This translates to ample online resources, tutorials, documentation, and forums where beginners and experienced developers can find help and connect with others.\n**4. Extensive libraries and frameworks:** Python boasts a rich ecosystem of powerful libraries and frameworks that cater to various tasks and doma

In [ ]:
await model.ainvoke(message)

"## Pros of Python:\n\n* **Easy to learn and read:** Python has a simple and intuitive syntax, making it a great choice for beginners and experienced programmers alike.\n* **Versatile:** Python can be used for a wide range of tasks, including web development, data science, machine learning, automation, and more.\n* **Large and active community:** Python has a vast and active community of developers, which means there are plenty of resources available to help you learn and solve problems.\n* **Extensive libraries and frameworks:** Python has a rich ecosystem of libraries and frameworks that can be used to accomplish almost any task.\n* **Cross-platform:** Python runs on all major operating systems, making it a portable and convenient choice.\n* **Free and open-source:** Python is free to use and distribute, making it accessible to everyone.\n* **Interpreted:** Python code is executed line by line, which makes it easier to debug and test.\n\n## Cons of Python:\n\n* **Dynamically typed:**

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_google_vertexai import VertexAI
import json  # For formatting the output as JSON
from datetime import datetime
from elasticsearch import Elasticsearch

# Define the Bedrock model with specific parameters (temperature, max tokens)
llm = VertexAI(model_name="gemini-1.0-pro-002")

# Create a Pydantic data model for the expected response structure
class TechAdvancementSummary(BaseModel):
    topic: str = Field(description="The topic under discussion, e.g., 'Agentic AI'")
    summary: str = Field(description="A summarized description of advancements for the topic")

# Define a query asking for a summary of advancements in a specific topic
tech_query = "Summarize the advancements in Agentic AI."

# Set up a JSON output parser to validate and parse the model response into the Pydantic structure
parser = JsonOutputParser(pydantic_object=TechAdvancementSummary)

# Create a prompt template to structure the query and ensure correct formatting of the output
prompt = PromptTemplate(
    template="Provide a summary for the following topic.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},  # Format instructions for JSON
)

# Create a chain that ties together the prompt, model (LLM), and parser
chain = prompt | llm | parser

# Invoke the chain to get the structured response as a dictionary
response = chain.invoke({"query": tech_query})

# Format the response as a JSON string and print it in a readable format
print(json.dumps(response, indent=2))

# Elasticsearch setup
es = Elasticsearch(
    api_key="useyours",
    cloud_id="useyours"
)

# Prepare the data for Elasticsearch
doc = {
    "timestamp": datetime.now().isoformat(),
    "prompt_template": {
        "input_variables": prompt.input_variables,
        "partial_variables": prompt.partial_variables,
        "template": prompt.template
    },
    "output": response  # Store model output
}

# Index the document into Elasticsearch
es.index(index="ai-responses-gemini", document=doc)

# Display the result
print("Response indexed in Elasticsearch successfully!")

{
  "topic": "Agentic AI",
  "summary": "Agentic AI is a research area that aims to endow artificial agents with a sense of agency, allowing them to act autonomously and make their own decisions. This is in contrast to traditional AI systems, which are typically rule-based or reactive. Some examples of agentic AI systems include self-driving cars, personal assistants, and autonomous robots. There are a number of potential benefits to developing agentic AI systems. For example, they could be used to automate tasks that are currently performed by humans, such as driving or customer service. They could also be used to create new products and services that are not possible with traditional AI systems. However, there are also some potential risks associated with agentic AI. For example, there is a risk that agentic AI systems could become too powerful and independent, leading to a loss of human control. Additionally, there is a risk that agentic AI systems could be used for malicious purpos